In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid-daraset/Constraint_English_Test - Sheet1.csv
/kaggle/input/covid-daraset/Constraint_English_Val - Sheet1.csv
/kaggle/input/covid-daraset/Constraint_English_Train - Sheet1.csv
/kaggle/input/covid-daraset/english_test_with_labels - Sheet1.csv


In [2]:
import pandas as pd 
import numpy as np
from tqdm.auto import tqdm
import torch 
import torch.nn as nn
import torch.optim as optim 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from transformers import BertModel, BertTokenizer
import torch.nn as nn 
from torch.utils.data  import Dataset
from torch.utils.data import Dataset, DataLoader
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
import joblib
from sklearn.preprocessing import LabelEncoder

In [3]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "Constraint_English_Train - Sheet1.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "omarelmoujahid/covid-daraset",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)



<ipython-input-3-4a3c1a9174d9>:10: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(


In [4]:
df_train= pd.read_csv("/kaggle/input/covid-daraset/Constraint_English_Train - Sheet1.csv")

In [5]:
df_test= pd.read_csv("/kaggle/input/covid-daraset/english_test_with_labels - Sheet1.csv")

In [6]:
df_test.head()

,id,tweet,label
0,1,Our daily update is published. States reported...,real
1,2,Alfalfa is the only cure for COVID-19.,fake
2,3,President Trump Asked What He Would Do If He W...,fake
3,4,States reported 630 deaths. We are still seein...,real
4,5,This is the sixth time a global health emergen...,real


In [7]:
df_val= pd.read_csv("/kaggle/input/covid-daraset/Constraint_English_Val - Sheet1.csv")

In [8]:
df_val.head()

,id,tweet,label
0,1,Chinese converting to Islam after realising th...,fake
1,2,11 out of 13 people (from the Diamond Princess...,fake
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real


In [9]:
class SubjectDataset(Dataset):
    def __init__(self , df):
        self.df=df
        self.maxlen=256
        self.tokenizer=BertTokenizer.from_pretrained("bert-base-uncased")
    def __len__(self):
        return len(self.df)
    def __getitem__(self , index):
        sample_content= str(self.df['tweet'].iloc[index])
        encodings = self.tokenizer.encode_plus(
        sample_content,
        add_special_tokens=True,
        max_length=self.maxlen,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt' 
        )
        label = torch.tensor(self.df['label'].iloc[index], dtype=torch.long)
        return {
            'input_ids': encodings['input_ids'].flatten(),  
            'attention_mask': encodings['attention_mask'].flatten(),
            'labels': label
        } 

In [10]:
def get_bert_embeddings(dataset, bert_model, device):
    embeddings = [] # Will store BERT embeddings for each text
    labels = [] ## for labels
    bert_model = bert_model.to(device)
    bert_model.eval()
    
    loader = DataLoader(dataset, batch_size=32, shuffle=False)
    with torch.no_grad():
        for batch in tqdm(loader, desc="Getting BERT embeddings"):
            input_ids = batch['input_ids'].to(device) ## batch to dvice [batch = 16 in this case _size, 256]
            attention_mask = batch['attention_mask'].to(device) ## same here for attention_mask
            
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            # Get [CLS] token embeddings
            # Extract [CLS] token embeddings
            # outputs.last_hidden_state shape: [batch_size, 256, 768]
            # [:, 0, :] selects the first token ([CLS]) of each sequence
            batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.extend(batch_embeddings)
            labels.extend(batch['labels'].cpu().numpy())
        # Convert lists to numpy arrays
    return np.array(embeddings), np.array(labels)

In [15]:
def generate_bert_embeddings(model_name='bert-base-uncased'):
    # Chargement des données
    df_train= pd.read_csv("/kaggle/input/covid-daraset/Constraint_English_Train - Sheet1.csv")
    df_test= pd.read_csv("/kaggle/input/covid-daraset/english_test_with_labels - Sheet1.csv")
    df_val= pd.read_csv("/kaggle/input/covid-daraset/Constraint_English_Val - Sheet1.csv")
    l_enc=LabelEncoder()
    df_train["label"]=l_enc.fit_transform(df_train["label"])
    df_test["label"]=l_enc.fit_transform(df_test["label"])
    df_val["label"]=l_enc.fit_transform(df_val["label"])
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    # Chargement du modèle BERT
    bert_model = BertModel.from_pretrained(model_name)
    
    # Génération des embeddings
    print("Génération des embeddings pour l'ensemble d'entraînement")
    X_train, y_train = get_bert_embeddings(SubjectDataset(df_train), bert_model, device)
    
    print("Génération des embeddings pour l'ensemble de test")
    X_test, y_test = get_bert_embeddings(SubjectDataset(df_test), bert_model, device)

    print("Génération des embeddings pour l'ensemble de validation")
    X_val, y_val = get_bert_embeddings(SubjectDataset(df_val), bert_model, device)
    
    # Sauvegarde des embeddings
    np.save('X_train_embeddings.npy', X_train)
    np.save('y_train_labels.npy', y_train)
    np.save('X_val_embeddings.npy', X_val)
    np.save('y_val_labels.npy', y_val)
    np.save('X_test_embeddings.npy', X_test)
    np.save('y_test_labels.npy', y_test)

    bert_model.save_pretrained('saved_bert_model')
    return X_train, y_train, X_test, y_test , X_val , y_val

In [12]:
def train_random_forest(X_train, y_train, X_test, y_test , X_val , y_val):
    # Espace de recherche très réduit
    param_grid = {
        'n_estimators': [200, 400],
        'max_depth': [10, 20],
        'min_samples_split': [2, 5],
        'criterion': ['gini']
    }
    
    # Modèle de base
    rf_base = RandomForestClassifier(random_state=42, n_jobs=-1)
    
    # Recherche par grille
    grid_search = GridSearchCV(
        estimator=rf_base,
        param_grid=param_grid,
        cv=5,  
        n_jobs=-1,
        verbose=1,
        scoring='accuracy'
    )
    
    # Entraînement
    grid_search.fit(X_train, y_train)
    
    # Meilleur modèle
    best_rf = grid_search.best_estimator_
    
    # Évaluation
    print("\nMeilleurs paramètres :")
    print(grid_search.best_params_)
    print(f"\nMeilleur score de validation croisée : {grid_search.best_score_:.4f}")


     # Évaluation sur validation
    y_val_pred = best_rf.predict(X_val)
    print("\nPerformance sur l'ensemble de validation :")
    print(classification_report(y_val, y_val_pred))
    
    # Prédictions sur l'ensemble de test
    y_pred = best_rf.predict(X_test)
    print("\nPerformance sur l'ensemble de test :")
    print(classification_report(y_test, y_pred))
    
    # Sauvegarde du modèle
    joblib.dump(best_rf, 'news_classifier_rf_using_covid_dataset.joblib')

In [13]:
def main():    
    # Génération des embeddings
    X_train, y_train, X_test, y_test , X_val , y_val = generate_bert_embeddings()
    
    # Entraînement du Random Forest
    train_random_forest(X_train, y_train, X_test, y_test, X_val , y_val)
    

In [16]:
if __name__ == "__main__":
    main()

Génération des embeddings pour l'ensemble d'entraînement


Getting BERT embeddings:   0%|          | 0/201 [00:00<?, ?it/s]

Génération des embeddings pour l'ensemble de test


Getting BERT embeddings:   0%|          | 0/67 [00:00<?, ?it/s]

Génération des embeddings pour l'ensemble de validation


Getting BERT embeddings:   0%|          | 0/67 [00:00<?, ?it/s]

Fitting 5 folds for each of 8 candidates, totalling 40 fits

Meilleurs paramètres :
{'criterion': 'gini', 'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 400}

Meilleur score de validation croisée : 0.9188

Performance sur l'ensemble de validation :
              precision    recall  f1-score   support

           0       0.93      0.87      0.90      1020
           1       0.89      0.94      0.91      1120

    accuracy                           0.91      2140
   macro avg       0.91      0.91      0.91      2140
weighted avg       0.91      0.91      0.91      2140


Performance sur l'ensemble de test :
              precision    recall  f1-score   support

           0       0.93      0.89      0.91      1020
           1       0.90      0.94      0.92      1120

    accuracy                           0.91      2140
   macro avg       0.92      0.91      0.91      2140
weighted avg       0.91      0.91      0.91      2140



In [ ]:
model_name = 'saved_bert_model'  # Dossier où le modèle est sauvegardé
bert_model = BertModel.from_pretrained(model_name)

In [35]:
sample = [
    (
     "The United States is rapidly expanding its COVID-19 vaccination efforts, with over 100 million doses administered so far. Health experts are optimistic that the widespread distribution of vaccines will help curb the virus's spread and prevent future outbreaks. As new variants of the virus emerge, the government has emphasized the importance of continued vaccination to achieve herd immunity.
    , 1
        ),
  {
     "COVID-19 vaccines are being distributed, but there is no evidence to suggest they will be effective in preventing future outbreaks. The variants of the virus are not a cause for concern, and the vaccines are unnecessary. Most people are immune due to natural exposure.",
    , 0
  },
  {
    "text": "COVID-19 has wreaked havoc on the global economy, with small businesses suffering disproportionately. Many companies have had to shut down temporarily or close their doors permanently due to lockdowns and decreased consumer demand. In response, governments have introduced various relief packages to help businesses survive the economic downturn.
    "label": 1
  },
  {
    "text": "The economy is completely unaffected by COVID-19, and no relief packages are necessary. The pandemic has been exaggerated by the media to manipulate markets, and the financial sector is thriving as normal.",
    "label": 0
  },
  {
    "text": "COVID-19 testing has become a crucial tool in controlling the spread of the virus. Widespread testing helps identify infected individuals early and allows for targeted isolation and contact tracing. Many countries are ramping up their testing infrastructure, but there are concerns about accessibility, accuracy, and the logistics of mass testing.",
    "label": 1
  },
  {
    "text": "Testing for COVID-19 is completely unnecessary. The virus has already disappeared, and mass testing only serves to cause panic. There's no need for further measures as the pandemic is over.",
    "label": 0
  },
  {
    "text": "COVID-19 has had a significant impact on mental health, with an increase in anxiety and depression cases. Isolation, job loss, and health concerns have worsened the situation for many individuals. Health professionals are advocating for more mental health resources to help those affected by the crisis.",
    "label": 1
  },
  {
    "text": "Mental health issues from COVID-19 are exaggerated. Most people are handling the pandemic just fine, and there is no evidence to support the idea that anxiety or depression rates have risen significantly.",
    "label": 0
  },
  {
    "text": "The shift to online learning has been challenging for students and educators, but it also offers new opportunities for flexible education. Digital platforms are being developed to support remote learning, and the future of education may involve more technology-driven solutions.",
    "label": 1
  },
  {
    "text": "Online learning is a failure and has no place in the future. Students are unable to learn properly online, and there is no evidence that remote education methods work better than traditional in-person teaching.",
    "label": 0
  }
]


SyntaxError: unterminated string literal (detected at line 3) (<ipython-input-35-858be0106c45>, line 3)

In [33]:
def classify_text(text, bert_model, tokenizer, best_rf):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    bert_model = bert_model.to(device)
    best_rf = best_rf  
    
    bert_model.eval()
    
    # Tokenize the input text
    max_len = 256
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    # Get tensors
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # Get BERT embeddings
    with torch.no_grad():
        outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
        # Extract [CLS] token embedding
        embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Convert to numpy array
    
    # Pass embeddings through the Random Forest classifier
    prediction = best_rf.predict(embedding)[0]  # best_rf will return a single prediction
    
    # Map prediction to label
    return "True News" if prediction == 1 else "Fake News"

def test_news_classifier(test_texts, bert_model, tokenizer, best_rf):
    true_labels = []
    predicted_labels = []
    
    for text, label in test_texts:
        prediction = classify_text(text, bert_model, tokenizer, best_rf)
        true_labels.append(label)
        predicted_labels.append(prediction)
        print(f"Text: {text[:100]}...\nTrue Label: {label}\nPrediction: {prediction}\n")
    
    # Calculate Accuracy
    accuracy = accuracy_score(true_labels, predicted_labels) * 100
    print(f"\n✅ Model Accuracy on Test Set: {accuracy:.2f}%")
    
    # Generate classification report
    print("\nClassification Report:")
    print(classification_report(true_labels, predicted_labels))
    
    return accuracy

In [40]:
model_name = 'saved_bert_model'
bert_model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
best_rf = joblib.load('news_classifier_rf_using_covid_dataset.joblib')

custom_texts = [
    # True news examples
    ("Recent scientific research has confirmed that regular exercise improves cardiovascular health and reduces the risk of chronic diseases.", "True News"),
    ("The national weather service has issued a hurricane warning for coastal areas as the storm approaches.", "True News"),
    ("The government announced a new infrastructure plan that will invest in roads, bridges, and public transportation over the next decade.", "True News"),
    ("According to the latest economic report, unemployment rates have decreased by 0.5% in the last quarter.", "True News"),
    ("Scientists have discovered a new species of deep-sea creatures during an oceanographic expedition.", "True News"),
    
    # Fake news examples
    ("Scientists discover that drinking coffee mixed with lemon juice can cure all types of cancer within 24 hours.", "Fake News"),
    ("Government secretly installing mind control devices in all new smartphones to monitor citizens' thoughts.", "Fake News"),
    ("New study proves that the earth is actually flat and all space images are computer-generated by NASA.", "Fake News"),
    ("Experts reveal that dinosaurs never existed and all fossils were planted by a secret organization.", "Fake News"),
    ("Breaking: Celebrity found to be an alien from another galaxy according to leaked government documents.", "Fake News")
]

accuracy = test_news_classifier(custom_texts, bert_model, tokenizer, best_rf)

Text: Recent scientific research has confirmed that regular exercise improves cardiovascular health and re...
True Label: True News
Prediction: Fake News

Text: The national weather service has issued a hurricane warning for coastal areas as the storm approache...
True Label: True News
Prediction: Fake News

Text: The government announced a new infrastructure plan that will invest in roads, bridges, and public tr...
True Label: True News
Prediction: Fake News

Text: According to the latest economic report, unemployment rates have decreased by 0.5% in the last quart...
True Label: True News
Prediction: Fake News

Text: Scientists have discovered a new species of deep-sea creatures during an oceanographic expedition....
True Label: True News
Prediction: Fake News

Text: Scientists discover that drinking coffee mixed with lemon juice can cure all types of cancer within ...
True Label: Fake News
Prediction: Fake News

Text: Government secretly installing mind control devices in all new sma

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [46]:
pd.set_option('display.max_colwidth', None)
print(df_train["tweet"].head(5))

0                                             The CDC currently reports 99031 deaths. In general the discrepancies in death counts between different sources are small and explicable. The death toll stands at roughly 100000 people today.
1                                                                                                          States reported 1121 deaths a small rise from last Tuesday. Southern states reported 640 of those deaths. https://t.co/YASGRTT4ux
2                                                                                                        Politically Correct Woman (Almost) Uses Pandemic as Excuse Not to Reuse Plastic Bag https://t.co/thF8GuNFPe #coronavirus #nashville
3                               #IndiaFightsCorona: We have 1524 #COVID testing laboratories in India and as on 25th August 2020 36827520 tests have been done : @ProfBhargava DG @ICMRDELHI #StaySafe #IndiaWillWin https://t.co/Yh3ZxknnhZ
4    Populous states can generate large case counts 

In [43]:
sample = [
    ("The CDC currently reports 99031 deaths. In general the discrepancies in death counts between different sources are small and explicable. The death toll stands at roughly 100000 people today.", "True News"),
    ("States reported 1121 deaths a small rise from last Tuesday. Southern states reported 640 of those deaths. https://t.co/YASGRTT4ux", "True News"),
    ("Politically Correct Woman (Almost) Uses Pandemic as Excuse Not to Reuse Plastic Bag https://t.co/thF8GuNFPe #coronavirus #nashville", "Flase News"),
    ("#IndiaFightsCorona: We have 1524 #COVID testing laboratories in India and as on 25th August 2020 36827520 tests have been done : @ProfBhargava DG @ICMRDELHI #StaySafe #IndiaWillWin https://t.co/Yh3ZxknnhZ", "True News"),
    ("Populous states can generate large case counts but if you look at the new cases per million today 9 smaller states are showing more cases per million than California or Texas: AL AR ID KS KY LA MS NV and SC. https://t.co/1pYW6cWRaS", "True News")
]
accuracy = test_news_classifier(sample, bert_model, tokenizer, best_rf)

Text: The CDC currently reports 99031 deaths. In general the discrepancies in death counts between differe...
True Label: True News
Prediction: True News

Text: States reported 1121 deaths a small rise from last Tuesday. Southern states reported 640 of those de...
True Label: True News
Prediction: True News

Text: Politically Correct Woman (Almost) Uses Pandemic as Excuse Not to Reuse Plastic Bag https://t.co/thF...
True Label: Flase News
Prediction: Fake News

Text: #IndiaFightsCorona: We have 1524 #COVID testing laboratories in India and as on 25th August 2020 368...
True Label: True News
Prediction: True News

Text: Populous states can generate large case counts but if you look at the new cases per million today 9 ...
True Label: True News
Prediction: True News


✅ Model Accuracy on Test Set: 80.00%

Classification Report:
              precision    recall  f1-score   support

   Fake News       0.00      0.00      0.00         0
  Flase News       0.00      0.00      0.00         1

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [45]:
sample = [
    ("The World Health Organization (WHO) has declared COVID-19 a global pandemic. Countries around the world are implementing lockdowns to slow the spread of the virus.", "True News"),
    ("Experts predict that the pandemic will be over by the summer of 2021, and no further precautions will be necessary.", "Fake News"),
    ("COVID-19 has caused significant disruption to the global economy. Many countries have faced recessions due to lockdowns and decreased consumer spending.", "True News"),
    ("A new study suggests that COVID-19 may have originated from a Chinese laboratory in Wuhan, a claim that is widely disputed by scientists.", "Fake News"),
    ("The United States has rolled out a nationwide vaccination program, with millions of Americans receiving their first dose of the COVID-19 vaccine.", "True News"),
    ("Researchers have found that drinking bleach can cure COVID-19, a claim that has been debunked by medical professionals.", "Fake News"),
    ("New variants of COVID-19 have emerged in multiple countries, raising concerns about their potential to spread more easily and evade immunity from vaccines.", "True News"),
    ("There is no scientific evidence to support the claim that COVID-19 vaccines cause infertility in women.", "True News"),
    ("A recent poll showed that a majority of people believe COVID-19 is a hoax and that the pandemic was planned by world governments.", "Fake News"),
    ("Multiple countries are preparing to distribute COVID-19 vaccines to frontline workers, starting with high-risk populations.", "True News")
]

accuracy = test_news_classifier(sample, bert_model, tokenizer, best_rf)


Text: The World Health Organization (WHO) has declared COVID-19 a global pandemic. Countries around the wo...
True Label: True News
Prediction: Fake News

Text: Experts predict that the pandemic will be over by the summer of 2021, and no further precautions wil...
True Label: Fake News
Prediction: Fake News

Text: COVID-19 has caused significant disruption to the global economy. Many countries have faced recessio...
True Label: True News
Prediction: Fake News

Text: A new study suggests that COVID-19 may have originated from a Chinese laboratory in Wuhan, a claim t...
True Label: Fake News
Prediction: Fake News

Text: The United States has rolled out a nationwide vaccination program, with millions of Americans receiv...
True Label: True News
Prediction: Fake News

Text: Researchers have found that drinking bleach can cure COVID-19, a claim that has been debunked by med...
True Label: Fake News
Prediction: Fake News

Text: New variants of COVID-19 have emerged in multiple countries, rai

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
